In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain

ProjDir = rel_path("..", "scripts", "12")

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col logpop, set log() for population data

In [2]:
d[:logpop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;

first(d, 5)

m12_6_1 = "
data{
    int total_tools[10];
    real logpop[10];
    int society[10];
}
parameters{
    real a;
    real bp;
    vector[10] a_society;
    real<lower=0> sigma_society;
}
model{
    vector[10] mu;
    sigma_society ~ cauchy( 0 , 1 );
    a_society ~ normal( 0 , sigma_society );
    bp ~ normal( 0 , 1 );
    a ~ normal( 0 , 10 );
    for ( i in 1:10 ) {
        mu[i] = a + a_society[society[i]] + bp * logpop[i];
        mu[i] = exp(mu[i]);
    }
    total_tools ~ poisson( mu );
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [3]:
stanmodel = Stanmodel(name="m12.6.1",  model=m12_6_1, num_warmup=2000,
num_samples=2000, output_format=:mcmcchain);

=====> /Users/rob/.julia/dev/StatisticalRethinking/notebooks/12


File /Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.1.stan will be updated.



Input data for cmdstan

In [4]:
m12_6_1_data = Dict("total_tools" => d[:total_tools], "logpop" => d[:logpop], "society" => d[:society]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_1_data, ProjDir, diagnostics=false, CmdStanDir=CMDSTAN_HOME);



--- Translating Stan model to C++ code ---
bin/stanc  --o=/Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.1.hpp /Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.1.stan
Model name=m12_6_1_model
Input file=/Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.1.stan
Output file=/Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.1.hpp
clang++ -std=c++1y -Wno-unknown-warning-option -Wno-tautological-compare -Wno-sign-compare      -O3 -I src -I stan/src -I stan/lib/stan_math/ -I stan/lib/stan_math/lib/eigen_3.3.3 -I stan/lib/stan_math/lib/boost_1.66.0 -I stan/lib/stan_math/lib/sundials_3.1.0/include    -DBOOST_RESULT_OF_USE_TR1 -DBOOST_NO_DECLTYPE -DBOOST_DISABLE_ASSERTS -DBOOST_PHOENIX_NO_VARIADIC_EXPRESSION     -c -MT /Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.1.o -MT /Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.1 -include /Users/rob/.julia/dev/StatisticalRethinking/notebooks/

Describe the draws

In [6]:
describe(chn)

Iterations = 1:2000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 2000

Empirical Posterior Estimates:
                   Mean            SD         Naive SE        MCSE         ESS   
         lp__  927.804683625  3.1446033940 0.03515773476 0.08154059227 1487.25019
accept_stat__    0.937985234  0.0852103038 0.00095268016 0.00138045491 2000.00000
   stepsize__    0.048896675  0.0054003108 0.00006037731 0.00060754446   79.00988
  treedepth__    5.919250000  0.4435696470 0.00495925942 0.01192510250 1383.56446
 n_leapfrog__   73.980500000 32.9372468116 0.36824961431 0.88458986957 1386.40675
  divergent__    0.000000000  0.0000000000 0.00000000000 0.00000000000        NaN
     energy__ -921.312971500  4.0367809698 0.04513258329 0.10130212506 1587.93697
            a    1.098911914  0.7465518959 0.00834670394 0.01472620103 2000.00000
           bp    0.261090511  0.0806247037 0.00090141159 0.00156796152 2000.00000
  a_society.1   -0.209696517  0.2507701641 0.00280369567 0.00466

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*